# Testing the code on simulated noise

This notebook provides tests of the `lalapps_pulsar_parameter_estimation_nested` code in comparison to the older `lalapps_pulsar_parameter_estimation` code when running on fake Gaussian noise. The former code uses [nested sampling](https://en.wikipedia.org/wiki/Nested_sampling_algorithm) to sample the likelihood, whilst the latter can either use MCMC sampling of the posterior, or compute the posterior explicitly over a grid of point in the parameter space.

To do the comparisons both codes are run such that they divide the data up into "chunks" containing 30 points each (or less if required at the end of the data), with the data assumed stationary over those 30 points. However, the `lalapps_pulsar_parameter_estimation_nested` can also be run in a way that allows it to actually calculate the length of stationary data chunks.

In [1]:
# import required modules

%matplotlib inline

from __future__ import print_function, division

import os
import sys

sys.path.append("/home/matthew/lscsoft/.virtualenvs/lalapps_knope_O2/lib/python2.7/site-packages")

import numpy as np
from scipy.signal import fftconvolve
import subprocess as sp

from time import time
import matplotlib as mpl
from matplotlib import pyplot as pl
import scotchcorner as sc

# these modules require lalapps
from lalapps.pulsarpputils import *

# some matplotlib configurations
mplparams = { \
      'backend': 'Agg',
      'text.usetex': True, # use LaTeX for all text
      'axes.linewidth': 0.5, # set axes linewidths to 0.5
      'axes.grid': True, # add a grid
      'grid.linewidth': 0.5,
      'font.family': 'sans-serif',
      'font.sans-serif': 'Avant Garde, Helvetica, Computer Modern Sans serif',
      'font.size': 15 }

mpl.rcParams.update(mplparams)

In [2]:
# define a function to work out how many effective (uncorrelated) samples the MCMC chains contain
def independent_samples(d):
    # get independent samples from the matrix d, where d contains an MCMC chain over a number of parameters.
    # The maximum autocorrelation length of all the parameter is used to define the number of effective samples.
    M = 5.
    K = 2.

    neff = []
    for par in d.T:
        x = par - np.mean(par)
        y = np.conj(x[::-1])
        z = fftconvolve(y, x, 'full')
        z = np.fft.ifftshift(z)
        acf = z[0:len(par)]
        acf = acf/acf[0]

        # estimate the autocorrelation length
        acf[1:] = acf[1:]*2.
        imax = int(len(acf)/K)
        cacf = np.cumsum(acf)
        s = np.arange(1, len(cacf)+1, dtype=np.float64)/float(M)
        idxs = np.flatnonzero(cacf[:imax] < s[:imax])
        if len(idxs) == 0:
            acl = 1
        else:
            acl = s[idxs[0]]
        neff.append(np.floor(len(par)/acl))

    minneff = int(np.min(neff))

    return d[np.random.permutation(d.shape[0])[0:minneff],:]

In [3]:
# set up the run directories
rundir = '.'
if not os.path.isdir(rundir): # make the directory
    os.makedirs(rundir)

detector = 'H1'        # the detector to use
psrname = 'J0000+0000' # a fake pulsar name

# set the output directory
outdir = os.path.join(rundir, 'output')
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# fake heterodyned data directory (for the lalapps_pulsar_parameter_estimation code
# this must be dataDET, where DET is e.g. H1)
datadir = os.path.join(rundir, 'data'+detector)
if not os.path.isdir(datadir): # make the directory
    os.makedirs(datadir)

# set the executables (this assumes that you are using virtual environments with virtualenvwrapper.sh and
# have a WORKON_HOME environment variable set, but you can change the path as required)
virenv = 'lalapps_knope_O2' # name of your virtual environment
execpath = os.path.join(os.environ['WORKON_HOME'], virenv)
execpath = os.path.join(execpath, 'bin')

ppenexec = os.path.join(execpath, 'lalapps_pulsar_parameter_estimation_nested')
n2pexec = os.path.join(execpath, 'lalapps_nest2pos') # script to convert nested samples to posterior samples
ppeexec = os.path.join(execpath, 'lalapps_pulsar_parameter_estimation')

## Running the codes

The code is run on random Gaussian noise with zero mean and a given standard deviation. I output a comparison plot between the posteriors produced for each realisation. I compare the upper limit produced from the nested-sampling-generated posterior and that calculated from the grid-based posterior. I also compare the run times for each implementation of the codes.

In [4]:
# set up some general inputs

# create a pulsar parameter (TEMPO-stype .par file) file format string
pardat = """PSRJ J0000+0000
RAJ 00:00:00.0
DECJ 00:00:00.0
F0 123.4567890
PEPOCH 56789.0
EPHEM DE405
"""

# output .par file containing right ascension and declination
parfile = os.path.join(rundir, 'pulsar.par')
fp = open(parfile, 'w')
fp.write(pardat)
fp.close()

# some defaults for the data generation
sigma = 1.0e-22        # set data standard deviation
dt = 60                # number of seconds between data points
gpsstart = 900000000   # GPS start time of data
duration = 864000      # duration of data (seconds) - 10 days
gpstimes = np.arange(gpsstart, gpsstart+duration, dt) # time stamps
dlen = len(gpstimes)   # length of data

# get an estimate of the 95% credible upper limit to be expected
ulest = 10.8*np.sqrt(sigma**2/dlen)

# create the prior file for the lalapps_pulsar_parameter_estimation_nested code
# (PHI0 in here is rotational phase, whereas for the older code it is GW phase for trixial emission l=m=2)
priorfile = os.path.join(rundir, 'pulsar.prior')
priordat = """H0 uniform 0 {}
PHI0 uniform 0 {}
PSI uniform {} {}
COSIOTA uniform -1 1
"""
fp = open(priorfile, 'w')
# set the h0 upper range to be 6 times the expected upper limit 
fp.write(priordat.format(6.*ulest, np.pi, -np.pi/4., np.pi/4.))
fp.close()

# lalapps_pulsar_parameter_estimation_nested run parameters
Nlive = '2048' # number of nested sample live points

# lalapps_pulsar_parameter_estimation (MCMC) run parameters
iterations = '250000' # number of MCMC iterations after burn-in
burnin = '100000'     # number of MCMC iterations for burn-in

# lalapps_pulsar_parameter_estimation (grid) run parameters
h0steps = '80' # number of grid points for each parameter
psisteps = '40'
phi0steps = '40'
cosiotasteps = '40'
h0max = '%.5e' % (6.*ulest) # maximum range of h0 values
h0ulc = '95'                # % credible h0 upper limit to output

In [6]:
# create data
data = sigma*np.random.randn(dlen, 2)

# append times and data together
tad = np.vstack((gpstimes, data.T)).T

# output fake data
datafile = os.path.join(datadir, 'finehet_'+psrname+'_'+detector)
np.savetxt(datafile, tad, fmt='%.6f %.7e %.7e', delimiter='\t')
    
# run lalapps_pulsar_parameter_estimation_nested
codecall = ' '.join([ppenexec, '--detectors', detector,
                     '--par-file', parfile, '--prior-file', priorfile,
                     '--input-files', datafile, '--outfile', os.path.join(outdir, 'fake_nest.hdf'),
                     '--Nlive', Nlive, '--Nmcmcinitial', '0', '--oldChunks'])

t0 = time()
p = sp.Popen(codecall, stdout=sp.PIPE, stderr=sp.PIPE, shell=True)
out, err = p.communicate()
t1 = time()
timenested = (t1-t0)

# nested samples need to be converted to posterior samples with lalapps_nest2pos
if os.path.isfile(os.path.join(outdir, 'fake_post.hdf')):
    os.remove(os.path.join(outdir, 'fake_post.hdf'))
codecall = ' '.join([n2pexec, '-p', os.path.join(outdir, 'fake_post.hdf'),
                     os.path.join(outdir, 'fake_nest.hdf')])
p = sp.Popen(codecall, stdout=sp.PIPE, stderr=sp.PIPE, shell=True)
out, err = p.communicate()

# get h0 upper limit from 
post, evsig, evnoise = pulsar_nest_to_posterior(os.path.join(outdir, 'fake_post.hdf'))
h0ul = upper_limit_greedy(post['h0'].samples, upperlimit=(float(h0ulc)/100.))
    
h0ulnested = (h0ul)
evratnested = (evsig-evnoise)
    
# run lalapps_pulsar_parameter_estimation in MCMC mode
codecall = ' '.join([ppeexec, '--detectors', detector,
                     '--pulsar', psrname, '--par-file', parfile, '--input-dir', rundir,
                     '--output-dir', outdir, '--mcmc', '--iterations', iterations,
                     '--burn-in', burnin, '--psi-bins', '1000', '--time-bins', '1440'])

t0 = time()
p = sp.Popen(codecall, stdout=sp.PIPE, stderr=sp.PIPE, shell=True)
out, err = p.communicate()
t1 = time()
timemcmc = (t1-t0)

# run lalapps_pulsar_parameter_estimation in grid mode
# delete any previously created evidence file as things get appended to it
evfile = os.path.join(outdir, 'evidence_%s' % psrname)
if os.path.isfile(evfile):
    os.remove(evfile)

codecall = ' '.join([ppeexec, '--detectors', detector,
                     '--pulsar', psrname, '--par-file', parfile, '--input-dir', rundir,
                     '--output-dir', outdir, '--psi-bins', '1000', '--time-bins', '1440',
                     '--h0steps', h0steps, '--maxh0', h0max, '--phi0steps', phi0steps,
                     '--psisteps', psisteps, '--cisteps', cosiotasteps, '--dob-ul', h0ulc])
    
t0 = time()
p = sp.Popen(codecall, stdout=sp.PIPE, stderr=sp.PIPE, shell=True)
out, err = p.communicate()
t1 = time()
timegrid = (t1-t0)

# read in evidence ratio and h0 upper limit produced by grid
evfile = os.path.join(outdir, 'evidence_%s' % psrname)
# evidence at end of first line, UL at end of second
fp = open(evfile, 'r')
evlines = fp.readlines()
h0ulgrid = float((evlines[1].split())[-1])
evratgrid = float((evlines[0].split())[-1])

# correct evidence and lalapps_pulsar_parameter_estimation does not apply the h0 and cos(iota) priors
# and also account for lalapps_pulsar_parameter_estimation using a 2pi phi0 range rather than pi
evratgrid = evratgrid - np.log(6.*ulest) - np.log(2.) + np.log(np.pi)

Read columns ['COSIOTA', 'H0', 'PHI0', 'PSI', 'deltalogL', 'deltaloglH1', 'logL', 'logPrior', 'logw', 'C22', 'CGW', 'F0', 'PHI0VECTOR', 'PHI22', 'PEPOCH', 'FREQNUM', 'PHI21', 'C21', 'DEC', 'PMRA', 'HPLUS', 'I31', 'Q22', 'HSCALARL', 'POSEPOCH', 'DIST', 'I21', 'PSITENSOR', 'IOTA', 'HSCALARB', 'PHI0TENSOR', 'HCROSS', 'PMDEC', 'COSTHETA', 'PX', 'RA', 'df', 'THETA', 'PSIVECTOR', 'F0_FIXED', 'HVECTORY', 'HVECTORX', 'PHI0SCALAR', 'PSISCALAR', 'LAMBDA']


In [7]:
# lalapps_pulsar_parameter_estimation_nested posterior samples
postppen, Z, N = pulsar_nest_to_posterior(os.path.join(outdir, 'fake_post.hdf'))
postppemcmc = np.loadtxt(os.path.join(outdir, 'MCMCchain_%s_%s' % (psrname, detector)), comments='%')

# get independent MCMC samples
postmcmcind = independent_samples(postppemcmc[:,1:])
ppemcmcidxs = {'h0': 0, 'phi0': 1, 'cosiota': 2, 'psi': 3}
print("Number of independent MCMC samples is %d from %s" % (postmcmcind.shape[0], postppemcmc.shape[0]))

Read columns ['COSIOTA', 'H0', 'PHI0', 'PSI', 'deltalogL', 'deltaloglH1', 'logL', 'logPrior', 'logw', 'C22', 'CGW', 'F0', 'PHI0VECTOR', 'PHI22', 'PEPOCH', 'FREQNUM', 'PHI21', 'C21', 'DEC', 'PMRA', 'HPLUS', 'I31', 'Q22', 'HSCALARL', 'POSEPOCH', 'DIST', 'I21', 'PSITENSOR', 'IOTA', 'HSCALARB', 'PHI0TENSOR', 'HCROSS', 'PMDEC', 'COSTHETA', 'PX', 'RA', 'df', 'THETA', 'PSIVECTOR', 'F0_FIXED', 'HVECTORY', 'HVECTORX', 'PHI0SCALAR', 'PSISCALAR', 'LAMBDA']
Number of independent MCMC samples is 4370 from 250000


In [8]:
print("Difference in upper limits:")
print("h0 upper limit (nested) = %.2e" % h0ulnested)
print("h0 upper limit (grid) = %.2e" % h0ulgrid)
print("percentage difference = %.2f" % (100.*np.abs(h0ulnested-h0ulgrid)/h0ulnested))
print("\n")
print("Difference in evidence ratios:")
print("log(evidence ratio) (nested) = %.2e" % evratnested)
print("log(evidence ratio) (grid) = %.2e" % evratgrid)


Difference in upper limits:
h0 upper limit (nested) = 2.28e-23
h0 upper limit (grid) = 2.28e-23
percentage difference = 0.20


Difference in evidence ratios:
log(evidence ratio) (nested) = -1.21e+00
log(evidence ratio) (grid) = -1.13e+00


In [ ]:
params = ['h0', 'phi0', 'cosiota', 'psi']

# lalapps_pulsar_parameter_estimation_nested posterior samples in known order
postppen2 = np.zeros((len(postppen['h0'].samples), len(params)))
postmcmc2 = np.zeros(postmcmcind.shape)
for i, p in enumerate(params):
    if 'phi0' in p:
        parscale = 2. # scale to GW phase
    else:
        parscale = 1.
    postppen2[:,i] = parscale*postppen[p].samples[:,0]
    
    # sort MCMC results into the same order as the nested samples
    postmcmc2[:,i] = postmcmcind[:,ppemcmcidxs[p]]

histops = {'histtype': 'step', 'color': 'b', 'edgecolor': 'b', 'linewidth': 1.5, 'alpha': 1.0}
contourops = {'colors': 'darkblue'}
labels = ['$h_0$', '$\phi_0$ (rads)', '$\cos{\iota}$', '$\psi$ (rads)']
limits = [(0., None), (0., 2.*np.pi), (-1., 1.), (-np.pi/4., np.pi/4.)]

postmcmcind = independent_samples(postppemcmc[:,1:])
ppemcmcidxs = {'h0': 0, 'phi0': 1, 'cosiota': 2, 'psi': 3}
print("Number of independent MCMC samples is %d from %s" % (postmcmcind.shape[0], postppemcmc.shape[0]))

# plot with scotchcorner
sp = sc.scotchcorner(postppen2, bins=20, ratio=2, labels=labels, datatitle='Nested sampling', showlims='both', hist_kwargs=histops,
                     showcontours=True, limits=limits, showpoints=True, scatter_kwargs={'alpha': 0.01},
                     contour_limits=limits, contour_levels=[0.5, 0.9], contour_kwargs=contourops)

histops = {'histtype': 'step', 'color': 'red', 'edgecolor': 'red', 'linewidth': 1.5, 'alpha': 1.0}
contourops = {'colors': 'darkred'}
sp.add_data(postmcmc2, hist_kwargs=histops, datatitle='MCMC', scatter_kwargs={'alpha': 0.01}, showcontours=True,
            contour_levels=[0.5, 0.9], limits=limits, contour_kwargs=contourops,contour_limits=limits)

# add grid based posteriors
for i, p in enumerate(['h0', 'phi', 'ciota', 'psi']):
    pdffile = os.path.join(outdir, 'pdf_%s.%s.%s' % (p, psrname, detector))
    pdfdata = np.loadtxt(pdffile)
    ax = sp.get_axis(labels[i])
    if i < 3:
        ax.plot(pdfdata[:,0], pdfdata[:,1], 'k--')
    else:
        ax.plot(pdfdata[:,1], pdfdata[:,0], 'k--')

sp.fig.tight_layout()
sp.savefig('simulatednoisetest.pdf')
sp.fig.savefig('simulatednoisetest.png', dpi=300)

Number of independent MCMC samples is 4370 from 250000


In [34]:
from scipy.stats import ks_2samp
for i, p in enumerate(params):
    D, pv = ks_2samp(postppen2[:,i], postmcmc2[:,i])
    print("{} KS-test p-value: {}".format(p, pv))

h0 KS-test p-value: 0.36011516063
phi0 KS-test p-value: 0.00146815173875
cosiota KS-test p-value: 0.647569187166
psi KS-test p-value: 0.00621619872567
